In [1]:
import requests
from bs4 import BeautifulSoup
from rich.progress import track
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import time
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json

declairce must have variable and constants

In [2]:
time_start = time.time()
SCOPES =  ['https://www.googleapis.com/auth/spreadsheets']


DOMAINS = 'https://lms.logika.asia/group?'
CPR_DOMAINS = 'https://lms.logika.asia/group/default/schedule'

date_now = date.today()
date_start = date_now - relativedelta(years=1)
date_end = date_now + relativedelta(years=1)

s = requests.Session()
try:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
except FileNotFoundError:
    flow = InstalledAppFlow.from_client_secrets_file(
        'credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
class_report = []
bases = ['NLB', "NVH", "PXL", "VCP"]
# bases = ["NLB"]
date = {"MON" : 2,"TUE" : 3,"WED" : 4,"THU" : 5,"FRI" : 6,"SAT" : 7,"SUN" : 8}

get percentage for each class happend the day before

In [3]:
def get_percentage():
  
  percentages = {}
  for base in bases:
    for i in range(1,3):
      date = date_now.today() - timedelta(i)
      para = f'?GroupWithLessonSearch%5BnextLessonTime%5D={date}+-+{date}&GroupWithLessonSearch%5BnextLessonNumber%5D=&GroupWithLessonSearch%5BnextLessonTitle%5D=&GroupWithLessonSearch%5Bid%5D=&GroupWithLessonSearch%5Btitle%5D={base}&GroupWithLessonSearch%5Bvenue%5D=&GroupWithLessonSearch%5Bactive_student_count%5D=&GroupWithLessonSearch%5Bweekday%5D=&GroupWithLessonSearch%5BlessonTeacherName%5D=&GroupWithLessonSearch%5Bteacher%5D=&GroupWithLessonSearch%5Bcurator%5D=&GroupWithLessonSearch%5Btype%5D=&GroupWithLessonSearch%5Bstatus%5D=&GroupWithLessonSearch%5BlessonFormat%5D=&GroupWithLessonSearch%5Bis_online%5D=&export=true&name=default&exportType=html'
      header =  {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7",
        "cache-control": "max-age=0",
        "sec-ch-ua": "\"Chromium\";v=\"116\", \"Not)A;Brand\";v=\"24\", \"Google Chrome\";v=\"116\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "cookie": "_ym_uid=1664966280852805697; intercom-device-id-ufjpx6k3=fd3080c1-5a63-4af6-bd1f-15268d549a7e; userId=30454; _grid_page_size_schedule=35d0980fa38e2255112d0c62698773cab8aa12a81c6735caf172064b5eb6ea47a%3A2%3A%7Bi%3A0%3Bs%3A24%3A%22_grid_page_size_schedule%22%3Bi%3A1%3Bs%3A3%3A%22200%22%3B%7D; _grid_page_size=d3ebbdf9ec9235bfc4ba59572a6d3dd403a563222be148e9c705e91612194e49a%3A2%3A%7Bi%3A0%3Bs%3A15%3A%22_grid_page_size%22%3Bi%3A1%3Bs%3A3%3A%22200%22%3B%7D; _ym_d=1680775536; studentId=1700942; studentAccessToken=59839bd5ef574b15cd869fa67e04fcdf4a97e278085dee7a21dfeb1f5a33cd62; studentCreatedTimestamp=1693481460; previousPage=%2F; createdTimestamp=1693978386; accessToken=6f7fa0398bac5ce10b0d85e00fc01725eae9b60c7188cfc1963c98ed73504e82; SERVERID=b440; _backendMainSessionId=81389387a907ff2840bad3c150bb2e76; _gid=GA1.2.1044467574.1693978385; _ym_isad=2; _ym_visorc=w; _ga=GA1.2.1620969847.1664966279; _ga_3QSGZBLTE3=GS1.1.1693978384.233.1.1693978511.59.0.0; _ga_4CL98HXNQK=GS1.1.1693978385.60.1.1693978530.0.0.0",
        "Referer": f"{CPR_DOMAINS}?GroupLessonSearch%5Bstart_time%5D={date}%20-%20{date}&GroupLessonSearch%5Bnumber%5D=&GroupLessonSearch%5Blesson.title%5D=&GroupLessonSearch%5Bgroup_id%5D=&GroupLessonSearch%5Bgroup.title%5D={base}&GroupLessonSearch%5Bgroup.active_student_count%5D=&GroupLessonSearch%5Bweekday%5D=&GroupLessonSearch%5Bteacher.name%5D=&GroupLessonSearch%5Bgroup.teacher.name%5D=&GroupLessonSearch%5Bis_online%5D=",
        "Referrer-Policy": "strict-origin-when-cross-origin"
      }
      end_point = CPR_DOMAINS + para
      res = s.get(end_point, headers = header)
      output = res.content.decode('utf-8')
      soup = BeautifulSoup(output, 'html.parser', from_encoding='utf-8')
      soup.find('tbody')
      # get data 
      all_tr = soup.find_all('tr')
      for i in range(len(all_tr)):
        tr = all_tr[i]
        all_td = tr.find_all('td')
        next_les = all_td[1]
        next_les = next_les.string
        group_title = all_td[4].find('a')
        
        id = all_td[4].find('p')
        class_occurrences = all_td[5].find('span')
        if class_occurrences is None:
          continue
        class_occurrences = class_occurrences.string
        if group_title is None:
          continue
        # print(str(class_occurrences))
        group_title = group_title.string
        percentages[str(group_title)] = [str(class_occurrences), int(next_les)]
  
  return percentages 
percentages = get_percentage()
print(percentages)

c:\Users\hazaz\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


{'[Campus]_PTSY1.[No]_[Day]_[Format]': ['0%', 8], '[Campus]_PTPY1.[No]_[Day]_[Format]': ['0%', 8], 'NVH_GD39_TUE_OFF (PRIVATE)': ['0%', 28], 'NLB_GD51_WED_ONL': ['100%', 17], 'NEW PTSY2 Schedule Template - TAMMY USE': ['0%', 26], 'NEW PTSY1 Schedule Template - TAMMY USE': ['0%', 24], 'NVH_GD42_TUE_OFF': ['100%', 24], 'PXL_GD21_TUE_OFF': ['100%', 10], 'NVH_CK37_TUE_OFF': ['0%', 13], 'VCP_PTSY1.14_TUE_OFF': ['100%', 19], 'NVH_GD62_TUE_OFF': ['134%', 3], '[Campus]_GD.[No]_[Day]_[Format]': ['0%', 8], '[Campus]_VP.[No]_[Day]_[Format]': ['0%', 8], '[Campus]_CK.[No]_[Day]_[Format]': ['0%', 8], 'NVH_GD53_TUE_OFF': ['100%', 10], 'NLB_PTSY2.24_TUE_OFF': ['100%', 13], 'VCP_PTSY2.5_TUE_OFF (PRIVATE)': ['100%', 29], 'NVH_PTPY2.8_TUE_ONL': ['100%', 32], 'VCP_WDY1.5_TUE_ONL': ['100%', 11], 'NLB_VP34_TUE_OFF': ['67%', 27], 'NVH_VP35_TUE_OFF': ['100%', 24], 'NEW Python Pro Year 1 Schedule Template - TAMMY USE': ['0%', 26], 'NLB_CK40_TUE_OFF': ['100%', 1], 'VCP_GD28_TUE_OFF': ['75%', 5], 'PXL_CK12_TUE_O

main function 
will get update in the future and clean up soon.

In [4]:
def get_data(percentages):
  feedback_need = {}
  global nlb_groups, nvh_groups, pxl_groups, vcp_groups
  wday = ''
  name_PT = ''
  for base in bases:
      new_start_time = time.time()
      para = f'GroupSearch%5Bid%5D=&GroupSearch%5Btitle%5D={base}&GroupSearch%5Bvenue%5D=&GroupSearch%5Bactive_student_count%5D=&GroupSearch%5BnextLessonTime%5D=&GroupSearch%5BnextLessonNumber%5D=&GroupSearch%5BnextLessonTitle%5D=&GroupSearch%5Bteacher%5D=&GroupSearch%5Bcurator%5D=&GroupSearch%5Btype%5D=&GroupSearch%5Btype%5D%5B%5D=regular&GroupSearch%5Btype%5D%5B%5D=intensive&GroupSearch%5Bstatus%5D=&GroupSearch%5BcourseContentType%5D=&GroupSearch%5BcourseContentType%5D%5B%5D=course&GroupSearch%5BcourseContentType%5D%5B%5D=intensive&GroupSearch%5BcourseStartTime%5D={date_start}+-+{date_now}&GroupSearch%5BcourseEndTime%5D={date_now}+-+{date_end}&presetType=active&export=true&name=default&exportType=html'
      header = {

          "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
          "accept-language": "en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7",
          "cache-control": "max-age=0",
          "sec-ch-ua": "\"Chromium\";v=\"116\", \"Not)A;Brand\";v=\"24\", \"Google Chrome\";v=\"116\"",
          "sec-ch-ua-mobile": "?0",
          "sec-ch-ua-platform": "\"Windows\"",
          "sec-fetch-dest": "document",
          "sec-fetch-mode": "navigate",
          "sec-fetch-site": "same-origin",
          "sec-fetch-user": "?1",
          "upgrade-insecure-requests": "1",
          "cookie": "_ym_uid=1664966280852805697; intercom-device-id-ufjpx6k3=fd3080c1-5a63-4af6-bd1f-15268d549a7e; userId=30454; _grid_page_size_schedule=35d0980fa38e2255112d0c62698773cab8aa12a81c6735caf172064b5eb6ea47a%3A2%3A%7Bi%3A0%3Bs%3A24%3A%22_grid_page_size_schedule%22%3Bi%3A1%3Bs%3A3%3A%22200%22%3B%7D; _grid_page_size=d3ebbdf9ec9235bfc4ba59572a6d3dd403a563222be148e9c705e91612194e49a%3A2%3A%7Bi%3A0%3Bs%3A15%3A%22_grid_page_size%22%3Bi%3A1%3Bs%3A3%3A%22200%22%3B%7D; _ym_d=1680775536; studentId=1700942; studentAccessToken=59839bd5ef574b15cd869fa67e04fcdf4a97e278085dee7a21dfeb1f5a33cd62; studentCreatedTimestamp=1693481460; previousPage=%2F; createdTimestamp=1693978386; accessToken=6f7fa0398bac5ce10b0d85e00fc01725eae9b60c7188cfc1963c98ed73504e82; SERVERID=b440; _backendMainSessionId=81389387a907ff2840bad3c150bb2e76; _gid=GA1.2.1044467574.1693978385; _ym_isad=2; _ym_visorc=w; _ga=GA1.2.1620969847.1664966279; _ga_3QSGZBLTE3=GS1.1.1693978384.233.1.1693978511.59.0.0; _ga_4CL98HXNQK=GS1.1.1693978385.60.1.1693978530.0.0.0",
          "Referer": f"{DOMAINS}GroupSearch%5Bid%5D=&GroupSearch%5Btitle%5D={base}&GroupSearch%5Bvenue%5D=&GroupSearch%5Bactive_student_count%5D=&GroupSearch%5BnextLessonTime%5D=&GroupSearch%5BnextLessonNumber%5D=&GroupSearch%5BnextLessonTitle%5D=&GroupSearch%5Bteacher%5D=&GroupSearch%5Bcurator%5D=&GroupSearch%5Btype%5D=&GroupSearch%5Btype%5D%5B%5D=regular&GroupSearch%5Btype%5D%5B%5D=intensive&GroupSearch%5Btype%5D%5B%5D=individual&GroupSearch%5Bstatus%5D=&GroupSearch%5BcourseContentType%5D=&GroupSearch%5BcourseContentType%5D%5B%5D=course&GroupSearch%5BcourseContentType%5D%5B%5D=intensive&GroupSearch%5BcourseStartTime%5D={date_start}%20-%20{date_now}&GroupSearch%5BcourseEndTime%5D={date_now}%20-%20{date_end}&presetType=active",
          "Referrer-Policy": "strict-origin-when-cross-origin"
          }


      end_point = DOMAINS + para
      res = s.get(end_point, headers = header)
      output = res.content.decode('utf-8')

      soup = BeautifulSoup(output, 'html.parser', from_encoding='utf-8')
      soup.find('tbody')

      table_rows = soup.find_all('tr')
      groups = []
      feedback_need = {}
      # table_rows = list(table_rows)
      # print(len(table_rows))

      # table_rows.remove(table_rows[0])
      date_str = ''
      print(len(table_rows))
      for tr in track(table_rows, description= f"Processing {base}..."):
          lessons = []
          group = []
          all_td = tr.find_all('td')
          group_id = all_td[0].string
          group_name = all_td[1].find('a')
          group_name = str(group_name)[str(group_name).find('>')+1:str(group_name).find('</a>')]
          teacher_name = all_td[7].find('a')
          teacher_name = str(teacher_name)[str(teacher_name).find('>')+1:str(teacher_name).find('</a>')]
          for key in date.keys():
            if key in group_name:
                date_str = key
                break
          if date_str:
            wday = date[date_str]
          all_li = all_td[1].find('p')
          if group_name is None or str(group_name) == "NVH_UGD2_MON/WED_OFF (PRIVATE)":
                continue
          campus = all_td[2].string
          amount_student = all_td[3].string
          next_lesson_time = all_td[4].string
          next_lesson_time = str(next_lesson_time)
          next_lesson_index = all_td[5].string
          next_lesson_index = str(next_lesson_index)
          next_lesson_name = all_td[6].find('div')
          next_lesson_name = str(next_lesson_name)[str(next_lesson_name).find('>')+1:str(next_lesson_name).find('</div>')]   
          course_start_date = all_td[12].string
          course_end_date = all_td[13].string
          all_li = str(all_li)
          class_time = next_lesson_time[next_lesson_time.find(":")-2:]
          url = all_li[all_li.find("https"):all_li.find("</p>")]
          start = url.find("d/")
          end = url.find("/edit")
          id = url[start+2:end]
          print(group_name)
          if group_name == "NVH_CK38_FRI_OFF":
            continue  
          try:
              result = sheet.values().get(spreadsheetId=id, range='Report!AC3:AC40').execute()
              values = result.get('values', [])
             
              for j in values:
                  if j[0] != "#DIV/0!":
                      score = j[0]
                      lessons.append(round(float(score), 2)) 
                  else:
                      lessons.append(j[0])
                  
              if len(lessons) == 0:
                  result = sheet.values().get(spreadsheetId=id, range='Report!AE3:AE40').execute()
                  values = result.get('values', [])
                  
                  for j in values:
                      if j[0] != "#DIV/0!":
                          score = j[0]
                          lessons.append(float(score)) 
                      else:
                          lessons.append(j[0])
              while len(lessons) < 38:
                  lessons.append("#DIV/0!")
              # print(lessons)
              if next_lesson_index == "2" or next_lesson_index == "5" or next_lesson_index == "9" or next_lesson_index == "13" or next_lesson_index == "21" or next_lesson_index == "25" or next_lesson_index == "29":
                  feedback_need[group_name] = [id,int(next_lesson_index)]
              
              programT_name = sheet.values().get(spreadsheetId=id, range='Report!E6:M6').execute()
              values = programT_name.get('values', [])
              try:
                name_PT = str(values[0][0])
              except:
                name_PT = "None"
          except HttpError as err:
              time.sleep(10)
          all_li = str(all_li)
          
          url = all_li[all_li.find("https"):all_li.find("</p>")]
          try:
            try:
              group = [group_id, wday, str(group_name), str(teacher_name), name_PT , class_time, str(url), str(campus), int(amount_student[0]), next_lesson_time, int(next_lesson_index), percentages[str(group_name)][0], next_lesson_name, course_start_date, course_end_date]

            except:
              group = [group_id, wday, str(group_name), str(teacher_name), name_PT , class_time, str(url), str(campus), int(amount_student[0]), next_lesson_time, int(next_lesson_index), "TBU", next_lesson_name, course_start_date, course_end_date]
            # print(group_name)
            group.extend(lessons)
            groups.append(group)
            
          except ValueError as err:
            print(group_name, err)
          
      time_end = time.time()
      resource = {
          "majorDimension": "ROWS",
          "values": groups
      }
      cresult = sheet.values().clear(
          spreadsheetId="1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA",
          range=f"{base}!A2:ZZZ"
        ).execute()
      print(f"{base} has {len(groups)} groups  ")
      if base == "NLB":
        nlb_groups = groups
      if base == "NVH":
        nvh_groups = groups
      if base == "PXL":
        pxl_groups = groups
      if base == "VCP":
        vcp_groups = groups 
      # 1EzB0-CpBYUe1J8z9jjDEPIBc3tcOwyBJZyPXN6HPZgE for testing
      # 1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA for real data update

      result1 = sheet.values().update(
          spreadsheetId= "1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA",
          range= f"{base}!A2",
          valueInputOption="RAW",
          body=resource).execute()
  print("Done")
  return groups
groups = get_data(percentages)

Output()

Non

125


Non invalid literal for int() with base 10: 'S'

NLB_GD73_WED_OFF (PRIVATE)

...

VCP_GD14_MON_OFF

VCP_PTSY2.5_TUE_OFF (PRIVATE)

VCP_DL3_SAT_OFF

VCP_PTPY1.3_SAT_OFF

VCP has 49 groups  
Done


get the feedback from teacher use as a data point to train AI NLP model

In [5]:
def get_feedback():
  feedback_need,groups = get_data(percentages)
  student_sheet  = {2 : "C36:D45", 5 :"C36:D45", 9 : "C69:D78", 13 : "C102:D111", 21 : "C168:D177", 25 : "C201:D210", 29 : "C234:D243"}
  feedback_sheet = {2 : "M36:O45", 5 :"S36:V45" ,9 : "P69:S78", 13 : "P102:S111", 21 : "P168:S177", 25 : "P201:S210", 29 : "P234:S243"}
  # print(feedback_need)
  class_report = []
  
  for group_name,value in feedback_need.items():
  

    link = value[0]
    lesson = value[1]
    feed = feedback_sheet[lesson]
    student = student_sheet[lesson]
    try:
      result1 = sheet.values().get(spreadsheetId=link, range=f'Report!{student}').execute()
      values1 = result1.get('values', [])
      result2 = sheet.values().get(spreadsheetId=link, range=f'Report!{feed}').execute()
      values2 = result2.get('values', [])
      
      for i in range(len(values1)):
        try:
          class_report.append([group_name,lesson,values1[i][0],values2[i][0].replace("\n", " ").replace("\t", " ")])
        except:
          try:
            class_report.append([group_name,lesson,values1[i][0],"not yet"])
          except:
            pass
    except HttpError:
      time.sleep(60)

  return class_report
    
# get_feedback()

push to Feedback Sheet 

In [6]:
def push_to_sheet():
  # print(data)
  # 1EzB0-CpBYUe1J8z9jjDEPIBc3tcOwyBJZyPXN6HPZgE for testing
  # 1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA for real data update
  class_report = get_feedback()
  result = sheet.values().clear(
          spreadsheetId="1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA",
          range="Feedback Report!A1:ZZZ"
      ).execute()
  resource2 = {
        "majorDimension": "ROWS",
        "values": class_report
    }
  result = sheet.values().update(
            spreadsheetId= "1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA",
            range= f"Feedback Report!A2",
            valueInputOption="RAW",
            body=resource2).execute()
push_to_sheet()

Output()

Non

125


Non invalid literal for int() with base 10: 'S'

NLB_GD73_WED_OFF (PRIVATE)

NLB_CK40_TUE_OFF

NLB_VP50_WED_OFF

NLB_PTPY1.17_FRI_OFF

NLB_PTSY2.30_MON/SAT_ONL (PRIVATE)

KeyboardInterrupt: 

Get the Finished classes

In [ ]:
def fin_classes():
    fin_grp = []
    for base in bases:
        DOMAINS_FIN = "https://lms.logika.asia/group?"


        para = f"GroupSearch%5Bid%5D=&GroupSearch%5Btitle%5D={base}&GroupSearch%5Bvenue%5D=&GroupSearch%5Bactive_student_count%5D=&GroupSearch%5BnextLessonTime%5D=&GroupSearch%5BnextLessonNumber%5D=&GroupSearch%5BnextLessonTitle%5D=&GroupSearch%5Bteacher%5D=&GroupSearch%5Bcurator%5D=&GroupSearch%5Bstatus%5D=&GroupSearch%5Bstatus%5D%5B%5D=inactive&GroupSearch%5BcourseContentType%5D=&GroupSearch%5BcourseStartTime%5D=&GroupSearch%5BcourseEndTime%5D=&export=true&name=default&exportType=html"


        header = {

            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "accept-language": "en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7",
            "cache-control": "max-age=0",
            "sec-ch-ua": "\"Chromium\";v=\"116\", \"Not)A;Brand\";v=\"24\", \"Google Chrome\";v=\"116\"",
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "\"Windows\"",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-origin",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "cookie": "_ym_uid=1664966280852805697; intercom-device-id-ufjpx6k3=fd3080c1-5a63-4af6-bd1f-15268d549a7e; userId=30454; _grid_page_size_schedule=35d0980fa38e2255112d0c62698773cab8aa12a81c6735caf172064b5eb6ea47a%3A2%3A%7Bi%3A0%3Bs%3A24%3A%22_grid_page_size_schedule%22%3Bi%3A1%3Bs%3A3%3A%22200%22%3B%7D; _grid_page_size=d3ebbdf9ec9235bfc4ba59572a6d3dd403a563222be148e9c705e91612194e49a%3A2%3A%7Bi%3A0%3Bs%3A15%3A%22_grid_page_size%22%3Bi%3A1%3Bs%3A3%3A%22200%22%3B%7D; _ym_d=1680775536; studentId=1700942; studentAccessToken=59839bd5ef574b15cd869fa67e04fcdf4a97e278085dee7a21dfeb1f5a33cd62; studentCreatedTimestamp=1693481460; previousPage=%2F; createdTimestamp=1693978386; accessToken=6f7fa0398bac5ce10b0d85e00fc01725eae9b60c7188cfc1963c98ed73504e82; SERVERID=b440; _backendMainSessionId=81389387a907ff2840bad3c150bb2e76; _gid=GA1.2.1044467574.1693978385; _ym_isad=2; _ym_visorc=w; _ga=GA1.2.1620969847.1664966279; _ga_3QSGZBLTE3=GS1.1.1693978384.233.1.1693978511.59.0.0; _ga_4CL98HXNQK=GS1.1.1693978385.60.1.1693978530.0.0.0",
            "Referer": f"{DOMAINS_FIN}GroupSearch%5Bid%5D=&GroupSearch%5Btitle%5D={base}&GroupSearch%5Bvenue%5D=&GroupSearch%5Bactive_student_count%5D=&GroupSearch%5BnextLessonTime%5D=&GroupSearch%5BnextLessonNumber%5D=&GroupSearch%5BnextLessonTitle%5D=&GroupSearch%5Bteacher%5D=&GroupSearch%5Bcurator%5D=&GroupSearch%5Bstatus%5D=&GroupSearch%5Bstatus%5D%5B%5D=inactive&GroupSearch%5BcourseContentType%5D=&GroupSearch%5BcourseStartTime%5D=&GroupSearch%5BcourseEndTime%5D=&presetType=all",
            "Referrer-Policy": "strict-origin-when-cross-origin"
        }



        end_point = DOMAINS_FIN + para
        res = s.get(end_point, headers = header)
        output = res.content.decode('utf-8')
        soup = BeautifulSoup(output, 'html.parser', from_encoding='utf-8')
        soup.find('tbody')
        table_rows = soup.find_all('tr')
        
        table_rows.remove(table_rows[0])
        # print(table_rows)
        for tr in table_rows:
            all_td = tr.find_all('td')
            grp_id = str(all_td[0].string)
            grp_name = all_td[1].find("a")
            cpr_url = all_td[1].find("p")
            cpr_url = str(cpr_url)
            url = cpr_url[cpr_url.find("https"):cpr_url.find("</p>")]
            num_of_std = all_td[3].find("span")       
            num_of_std = str(num_of_std.string)[0]
            # if num_of_std == "N" or num_of_std == "0":
                # continue
            num_of_std = int(num_of_std)
            grp_name = str(grp_name.string) 
            group_status = all_td[9].string
            class_end_date = all_td[12].string
            if class_end_date:
                class_end_date = str(class_end_date)
                class_end_date = class_end_date[:class_end_date.find(" ")]
                # print(class_end_date)
                # class_end_date = date.strptime(class_end_date, "%d.%m.%Y")
                # class_end_date = class_end_date.strftime("%m.%d.%Y")
            else:
                class_end_date = None

            if str(group_status) == "Finished" and str(group_status) != "Collapsed" and (grp_name.find("Trial") == -1 or grp_name.find("Demo") != -1):
                fin_grp.append([grp_id , grp_name, num_of_std, url, group_status, class_end_date])   
    return fin_grp


fin_grp = fin_classes()
print(fin_grp)
resource = {
        "majorDimension": "ROWS",
        "values": fin_grp
    }

result = sheet.values().update(
    spreadsheetId= "1D8uRbAJXBJMcF0e8tXWKe5dRbiv-EW_by-SpNAxZ3qA",
    range= f"'Finished Groups'!A2",
    valueInputOption="RAW",
    body=resource).execute()

Testing with Database

In [ ]:
print(groups[:5])
data_to_insert = [tuple(row) for row in groups]
print(data_to_insert)
print(data_to_insert)

In [ ]:
import mysql.connector

# MySQL database configuration
config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'algo_db',
    'raise_on_warnings': True
}

# List of tuples containing data to insert


# Create a MySQL connection and cursor
connection = mysql.connector.connect(**config)
cursor = connection.cursor()

# Insert query with placeholders for each column
insert_query = """
INSERT INTO nlb
(Id, Week_Day, Class_Code, Teacher_name, Assigned_Tutor, Time, CPR, Campus, Student, Next_Lesson, Next_Lesson_Num, Attend_Percentage, Course, Start_Date, End_Date, L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12, L13, L14, L15, L16, L17, L18, L19, L20, L21, L22, L23, L24, L25, L26, L27, L28, L29, L30, L31, L32, L33, L34, L35, L36, L37, L38)
VALUES
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert data into the table
cursor.executemany(insert_query, data_to_insert)

# Commit the changes and close the cursor and connection
connection.commit()
cursor.close()
connection.close()